# <font color=red>LangChain:  Agents (with Tools)</font>
- https://docs.langchain.com/docs

## What Does LangChain Provide?
+ Models
  + embedding
  + LLM (e.g. OpenAI)
+ Prompts
  + prompt templates
  + few-shot
  + example-selectors
  + output parsers
+ Chains (a multi-step workflow composed of <em>links</em>)</br>
  + Links (one of: prompt, model, another chain)
+ Vector Database Access
  + Document Loaders
  + Text Splitting 
+ Memories (to facilitate chatbots or other 'iterative' sorts of apps)
<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
+ Agents (loop over Thought, Act, Observe)
  + Tools
    + math
    + web search
    + custom (user-defined)
</font></span>

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Agents (with Tools)
</font></span>
There are several types of agents in LangChain, some of which are still experimental.</br>
Here are some of the types of agents and agent-simulations currently implemented:
- Chatbots
- CAMEL Role-Playing Autonomous Cooperative Agents
- Generative Agents
- Simulated Environment: Gymnasium
- Two-Player (and Multi-player) Dungeons & Dragons
- Multi-agent authoritarian speaker selection
- Multi-agent Simulated Environment: Petting Zoo
- Agent Debates with Tools
- Plan-and-execute
- BabyAGI User Guide and BabyAGI with Tools
- CAMEL Role-Playing Autonomous Cooperative Agents
- Custom Agent with PlugIn Retrieval
- Plug-and-Play
- Multi-modal outputs: Image & Text
- SalesGPT - Your Context-Aware AI Sales Assistant With Knowledge Base
- Wikibase Agent

Agents iterate over this pattern until a solution is obtained or a max number of iterations is reached:</br>
<pre>
        Thought -> Action -> Observation -> 
</pre>
The agent uses Tools to solve some problem during the Action phase.</br>
Such tools may provide extra math or web-search capabilities.
Some of the tools are built-in to LangChain and some can be user-defined.
We will start with a small example using a built-in math tool (llm-math).

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
### Agent with built-in tool (<font color=green>llm-math</font>)
</font></span>

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=64)

tools = load_tools(["llm-math"], llm=llm)
# change verbose to False if you just want the answer
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run( "What is 12345678 to the 0.23 power ?" )

Now, let's add use of a tool that performs google web search, serpapi.</br>
Use of the serpapi tool is free, but you need to obtain a key to use it.

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
### Adding a second built-in tool (<font color=green>serpapi</font>)
</font></span>

In [ ]:
## you must have SERPAPI_API_KEY set in your environment
import os
# os.environ["SERPAPI_API_KEY"] = ""  ## fix this line if you do not have it in the environment

from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=64)

tools = load_tools(["llm-math", "serpapi"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run( "What is the age of the current USA president?  And what is his age raised to the 0.23 power?" )

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
### Making our own tool using <font color=green>StructuredTool</font>
</font></span>
<font color=green>StructuredTool</font> helps you to take a simple <em>function</em> and turn it into a tool.

In [ ]:
import os, re, time, openai, langchain

from typing import Optional, Union
from math import sqrt, cos, sin

from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool   # instead of BaseTool
from langchain.agents import initialize_agent, AgentType

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

def pythagoras_func(adjacent_side: Optional[Union[int, float]] = None,
                    opposite_side: Optional[Union[int, float]] = None,
                    angle: Optional[Union[int, float]] = None
                   ):
    """Hypotenuse calculator
       Use this tool when you need to calculate the length of an hypotenuse given one or two sides of a
       triangle and/or an angle (in degrees).  To use the tool you must provide at least two of the
       following parameters:  ['adjacent_side', 'opposite_side', 'angle'].
    """
    if adjacent_side and opposite_side:
        return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
    elif adjacent_side and angle:
        return adjacent_side / cos(float(angle))
    elif opposite_side and angle:
        return opposite_side / sin(float(angle))
    else:
        return "Could not calculate the hypotenuse of the triangle."

pythagoras_tool = StructuredTool.from_function(pythagoras_func)

tools = [pythagoras_tool]

llm = ChatOpenAI(temperature=0.0, model_name='gpt-4')

agent = initialize_agent(
    # agent='chat-conversational-react-description',
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
)

system_msg_template = "You will help with problems such as figuring out values about sides of triangles."
system_msg_prompt = SystemMessagePromptTemplate.from_template(system_msg_template)
user_template = "Answer the following question: {question}"
user_msg_prompt = HumanMessagePromptTemplate.from_template(user_template)
chat_prompt = ChatPromptTemplate.from_messages( [system_msg_prompt,user_msg_prompt])

response = agent("If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?")
print(response)

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
### Making our own tool sub-classing <font color=green>BaseTool</font>
</font></span>
<font color=green>BaseTool</font> is a powerful super-class from which any tool can be built.</br>
Why we might prefer to use this instead of StructuredTool above is explained by the LangChain docs:</br>
<pre>
    If you want more control over the tool definition, you can subclass the BaseTool directly.
    For instance, maybe you want the api key to be loaded automatically from the environment variables.
</pre>
You might perform actions like loading the api key in the \_\_init\_\_ method of the new subclass.

In [ ]:
import os, re, time, openai, langchain

from typing import Optional, Union
from math import sqrt, cos, sin

from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool   # our tool will sub-class this
from langchain.agents import initialize_agent, AgentType

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

class PythagorasTool(BaseTool):
    name = "Hypotenuse calculator"
    description = "use this tool when you need to calculate the length of an hypotenuse " +\
                  "given one or two sides of a triangle and/or an angle (in degrees). " +\
                  "To use the tool you must provide at least two of the following parameters " +\
                  "['adjacent_side', 'opposite_side', 'angle']."

    def _run(
        self,
        adjacent_side: Optional[Union[int, float]] = None,
        opposite_side: Optional[Union[int, float]] = None,
        angle: Optional[Union[int, float]] = None
    ):
        if adjacent_side and opposite_side:
            return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
        elif adjacent_side and angle:
            return adjacent_side / cos(float(angle))
        elif opposite_side and angle:
            return opposite_side / sin(float(angle))
        else:
            return "Could not calculate the hypotenuse of the triangle."

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")


tools = [PythagorasTool()]

llm = ChatOpenAI(temperature=0.0, model_name='gpt-4')

agent = initialize_agent(
    # agent='chat-conversational-react-description',
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
)

system_msg_template = "You will help with problems such as figuring out values about sides of triangles."
system_msg_prompt = SystemMessagePromptTemplate.from_template(system_msg_template)
user_template = "Answer the following question: {question}"
user_msg_prompt = HumanMessagePromptTemplate.from_template(user_template)
chat_prompt = ChatPromptTemplate.from_messages( [system_msg_prompt,user_msg_prompt])

response = agent("If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?")
print(response)